<a href="https://colab.research.google.com/github/d43pan/AgentGPT/blob/main/figi_agent_01132025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Standard library imports
import os

# Third-party imports
import pandas as pd
from openfigipy import OpenFigiClient

# LlamaIndex imports
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.core.tools import FunctionTool

In [ ]:
# Configuration

OPEN_FIGI_APIKEY = 'your-figi-api-key'  # Replace with your API key
os.environ["OPENAI_API_KEY"] = 'your-openai-api-key'  # Replace with your API key

# Initialize clients
llm = LlamaOpenAI(model="gpt-4o-mini", temperature=0)
ofc = OpenFigiClient(api_key=OPEN_FIGI_APIKEY)
ofc.connect()


In [ ]:
def get_figis(tickers):
    """
    Get FIGIs for a list of ticker symbols.

    Args:
        tickers (list): List of ticker symbols

    Returns:
        list: List of corresponding FIGI identifiers
    """
    df_final = pd.DataFrame({
        'idType': ['TICKER'] * len(tickers),
        'idValue': tickers,
        'exchCode': ['US'] * len(tickers)
    })

    result = ofc.map(df_final)
    return result['figi'].tolist()

def get_figi_name_dict(keyword):
    """
    Search for securities matching a keyword and return FIGI to name mapping.

    Args:
        keyword (str): Search term for security names

    Returns:
        dict: Dictionary mapping FIGIs to security names
    """
    results = ofc.search(
        keyword,
        result_limit=10,
        marketSecDes='Equity',
        exchCode='US'
    )

    return {row['figi']: row['name'] for _, row in results.iterrows()}

def create_figi_agent(user_query: str):
    """
    Creates an agent that decides whether to use direct FIGI lookup by ticker
    or search by keyword based on the user's query.

    Args:
        user_query (str): User's question about finding FIGIs

    Returns:
        Response: Agent's response containing FIGI information
    """
    prompt = """
    You are a FIGI lookup expert. Your task is to analyze the user query and decide whether to:
    1. Use get_figis() when exact tickers are provided
    2. Use get_figi_name_dict() when searching by company name or keyword

    The two functions available to you are:

    1. get_figis(tickers: List[str]) -> List[str]
       - Use when you have exact ticker symbols
       - Input: List of ticker symbols (e.g., ["AAPL", "MSFT"])
       - Returns: List of corresponding FIGIs

    2. get_figi_name_dict(keyword: str) -> Dict[str, str]
       - Use when searching by company name or keyword
       - Input: Search keyword (e.g., "Apple" or "semiconductor")
       - Returns: Dictionary mapping FIGIs to company names

    Choose the most appropriate function based on whether the query contains:
    - Specific ticker symbols (use get_figis)
    - Company names or industry keywords (use get_figi_name_dict)
    """

    functions_list = [get_figis, get_figi_name_dict]
    agent = OpenAIAgent.from_tools(
        tools=[FunctionTool.from_defaults(fn=fn) for fn in functions_list],
        prefix_messages=[ChatMessage(role="system", content=prompt)],
        verbose=True,
        llm=LlamaOpenAI(model="gpt-4o-mini")
    )

    return agent.query(user_query)

In [ ]:
r1 = create_figi_agent("Find FIGIs for AAPL and MSFT")
print(r1)


Added user message to memory: Find FIGIs for AAPL and MSFT
=== Calling Function ===
Calling function: get_figis with args: {"tickers":["AAPL","MSFT"]}
Got output: ['BBG000B9XRY4', 'BBG000BPH459']

The FIGIs for AAPL and MSFT are as follows:
- AAPL: BBG000B9XRY4
- MSFT: BBG000BPH459


In [ ]:
response = create_figi_agent("Find FIGIs for SPDR etfs")
print(response)

Added user message to memory: Find FIGIs for SPDR etfs
=== Calling Function ===
Calling function: get_figi_name_dict with args: {"keyword":"SPDR ETF"}
Got output: {'BBG000PG8LL8': 'SPDR PORTFOLIO EUROPE ETF', 'BBG000RFRRV3': 'SPDR PORTFOLIO TIPS ETF', 'BBG000C9R1B2': 'SPDR NYSE TECHNOLOGY ETF', 'BBG000C9KLL4': 'SPDR GLOBAL DOW ETF', 'BBG01MZLS2N5': 'SPDR PORTFOLIO TREASURY ETF', 'BBG01PNQ84Y6': 'SPDR GALAXY HDAE ETF', 'BBG000D73SW9': 'SPDR S&P PHARMACEUTICALS ETF', 'BBG000KMKPY3': 'SPDR S&P BANK ETF', 'BBG000KMNWV8': 'SPDR S&P INSURANCE ETF', 'BBG000MQVF45': 'SPDR S&P SEMICONDUCTOR ETF'}

Here are the FIGIs for SPDR ETFs:

- **SPDR PORTFOLIO EUROPE ETF**: BBG000PG8LL8
- **SPDR PORTFOLIO TIPS ETF**: BBG000RFRRV3
- **SPDR NYSE TECHNOLOGY ETF**: BBG000C9R1B2
- **SPDR GLOBAL DOW ETF**: BBG000C9KLL4
- **SPDR PORTFOLIO TREASURY ETF**: BBG01MZLS2N5
- **SPDR GALAXY HDAE ETF**: BBG01PNQ84Y6
- **SPDR S&P PHARMACEUTICALS ETF**: BBG000D73SW9
- **SPDR S&P BANK ETF**: BBG000KMKPY3
- **SPDR S&P INSUR